Load google drive into the VM:

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Load labels from CamVid dataset:

In [0]:
import numpy as np

subdir_label_colors = "label_colorsSorted.txt"
directory = "drive/My Drive/datasets/camvid-master/"

labels = dict()
label_ids = []
with open(directory + subdir_label_colors, 'r') as color_file:
    file = color_file.read()
    i = 0
    for line in file.split('\n'):
      words = line.split()
      label_ids.append(words[0])
      labels[i] = np.array([words[1], words[2], words[3]]).astype(int)
      i += 1
#print(labels)
#print(label_ids)

Separate test, validation, and train data:

In [0]:
import torch
from PIL import Image
import numpy as np
import os
import pickle

directory = "drive/My Drive/datasets/camvid-master/"
subdir = "701_StillsRaw_full/"
subdir_l = "LabeledApproved_full/"
subdir_final_l = "Labels_full/"
train_test = 5
test_validation = 10
partition = {
    'train': [],
    'test': [],
    'validation': []
}


for root, dir, files in os.walk(directory + subdir):
  i = 0
  j = 0
  for file in files:
    if i % train_test == 0:
      partition['test'].append(file.split('.')[0])
    else:
      j += 1
      if j % test_validation == 0:
        partition['validation'].append(file.split('.')[0])
      else:
        partition['train'].append(file.split('.')[0])
    i += 1

Create new label files by replacing each pixel with corresponding class number:

In [0]:
for root, dir, files in os.walk(directory + subdir_l):
  for file in files:
    print(file)
    y = np.array(Image.open(directory + subdir_l + file))
    label = np.zeros((y.shape[0], y.shape[1]))
    for k, v in labels.items():
      label[np.where((y == np.asarray(v)).sum(-1) == 3)] = k
      
    try:
      os.stat(directory + subdir_final_l)
    except:
      os.mkdir(directory + subdir_final_l)
    pickle.dump(label, open(directory + subdir_final_l + file, 'wb'))
    
    '''
    for i, h in enumerate(y):
      for j, w in enumerate(h):
        for k, v in self.labels.items():
          if list(v) == list(y[i][j]):
            label[i][j] = k
    '''    

Create dataset class for loading train, test, and validation data:

In [0]:
import torch
from torch.utils import data
import pickle

class Dataset(data.Dataset):
  def __init__(self, list_IDs, labels, transform=None):
    self.labels = labels
    self.list_IDs = list_IDs
    self.transform = transform

  def __len__(self):
    return len(self.list_IDs)

  def __getitem__(self, index):
    ID = self.list_IDs[index]
    X = np.array(Image.open(directory + subdir + ID + '.png'))
    X = np.rollaxis(X, 2)
    if self.transform:
      X = self.transform(X)
    label = pickle.load(open(directory + subdir_final_l + ID + '_L.png', 'rb'))
#     for i, h in enumerate(y):
    return X, label

Initialize the network with specified parameters and train and test the network:

In [0]:
import torch.nn.functional as F
#from unet import UNet
from my_unet import UNet
import numpy as np
from torchsummary import summary

batch_size = 1

width = 960
height = 720

params = {'batch_size': batch_size,
          'shuffle': False,
          'num_workers': 6}

epochs = 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

training_set = Dataset(partition['train'], [])
training_generator = data.DataLoader(training_set, **params)

validation_set = Dataset(partition['validation'], [])
validation_generator = data.DataLoader(validation_set, **params)

test_set = Dataset(partition['test'], [])
test_generator = data.DataLoader(test_set, **params)

model = UNet(n_classes=50, padding=1, batch_norm=False, kernel_size=3, max_pool_or_stride=False).to(device)
#model  = UNet(n_class=50).to(device)
optim = torch.optim.Adam(model.parameters(), lr=1e-4)


print(summary(model, (3, height, width)))
train_loss = []
validation_loss = []
test_loss = []
accuracy = []


for epoch in range(epochs):
    i = 0
    epoch_loss = 0.0
    total_val_loss = 0.0
    total_test_loss = 0.0
    for x, y in training_generator:
      x, y = x.float() / 255.0, y.long()
      prediction = model(x.to(device))
      loss = F.cross_entropy(prediction, y.to(device))
      epoch_loss += loss.item()
      optim.zero_grad()
      loss.backward()
      optim.step()
      print('[%d, %d] Train loss = %.5f' %
          (epoch + 1, i + batch_size, loss.item()))
      i += batch_size
    train_loss.append(epoch_loss)
    
    for x, y in validation_generator:
      x, y = x.float() / 255.0, y.long()
      x, y = x.to(device), y.to(device)
      prediction = model(x)
      loss = F.cross_entropy(prediction, y)
      total_val_loss += loss.item()
    print('[%d] Validation loss = %.5f' % (epoch + 1, total_val_loss / len(validation_generator)))
    validation_loss.append(total_val_loss)
    
    acc = []
    for x, y in test_generator:
      x, y = x.float() / 255.0, y.long()
      x, y = x.to(device), y.to(device)
      prediction = model(x)
      loss = F.cross_entropy(prediction, y)
      total_test_loss += loss.item()
      acc.append(int((torch.argmax(prediction, dim=1) == y).sum()) / (width * height))
    print('[%d] Test loss = %.5f' % (epoch + 1, total_test_loss / len(test_generator)))
    test_loss.append(total_test_loss / len(test_generator))
    print('[%d] Test acc = %.5f' % (epoch + 1, np.array(acc).mean()))
    accuracy.append(np.array(acc).mean())


(For part I in section 1 of the project) Give a random image to the network and compare original label image with predicted image: 

In [0]:
from PIL import Image
import pickle

f = np.zeros((height, width, 3))
for i, h in enumerate(y[0]):
  for j, w in enumerate(h):
    for k, v in labels.items():
      if k == w:
        f[i][j] = v

pickle.dump(f, open(directory + 'test_image_orig.p', 'wb'))
f = Image.fromarray(np.array(f).astype('uint8'))
f.save(directory + 'test_img_orig.png')

y = np.rollaxis(prediction[0], 2)

pickle.dump(y, open(directory + 'test_image_pred.p', 'wb'))
q = np.zeros((height, width, 3))
for i, h in enumerate(y):
      for j, w in enumerate(h):
        for k, v in labels.items():
          if k == torch.argmax(w):
            q[i][j] = k

q = Image.fromarray(np.array(q).astype('uint8'))
q.save(directory + 'test_img_pred.png')

Draw loss and accuracy diagram:

In [0]:
import matplotlib.pyplot as plt

plt.plot([i + 1 for i in range(epochs)], [t / len(training_generator) for t in train_loss], marker='o')
plt.plot([i + 1 for i in range(epochs)], test_loss, marker='o')
plt.plot([i + 1 for i in range(epochs)], [t / len(validation_generator) for t in validation_loss], marker='o')
plt.title('Changes in loss function')
plt.xlabel('Epoch')
plt.ylabel('Cross Entropy Loss')
#plt.figure()
plt.savefig(directory + 'loss-4.jpg')
plt.show()

plt.plot([i + 1 for i in range(epochs)], accuracy, marker='o', color='red')
plt.title('Changes in accuracy')
plt.xlabel('Epoch')
plt.ylabel('accuracy')
#plt.figure()
plt.savefig(directory + 'acc-4.jpg')
plt.show()
